## 15.基于selenium与异步技术抓取微博

In [1]:
from selenium import webdriver #Selenium是一个用于Web应用程序测试的工具。Selenium测试直接运行在浏览器中，就像真正的用户在操作一样
import time 
import os
import csv
import pandas as pd

In [2]:
driver = webdriver.Chrome() #创建Chrome驱动
driver.get(r'https://weibo.com/') #获取微博网址
time.sleep(10)
driver.find_element_by_xpath(r'/html/body/div[1]/div[1]/div/div[1]/div/div/div[3]/div[2]/ul/li[3]/a').click() #找到相应区域模拟点击事件 使用xpath
# driver.find_element_by_xpath(r'//a[@class="S_txt1" and @node-type="loginBtn"]').click()
time.sleep(5)
#设置用户名和密码
driver.find_element_by_xpath(r'/html/body/div[4]/div[2]/div[3]/div[3]/div[1]/input').send_keys("myusername") 
# driver.find_element_by_xpath(r'//div[@class="item username input_wrap"]/input').send_keys("myusername")
time.sleep(3)
driver.find_element_by_xpath(r'/html/body/div[4]/div[2]/div[3]/div[3]/div[2]/input').send_keys("mypassword")
# driver.find_element_by_xpath(r'//div[@class="item password input_wrap"]/input').send_keys("mypassword")
time.sleep(3)
driver.find_element_by_xpath(r'/html/body/div[4]/div[2]/div[3]/div[3]/div[6]/a').click()
# driver.find_element_by_xpath(r'//a[@class="W_btn_a btn_34px"]').click()
time.sleep(3)

In [4]:
#同上
driver = webdriver.Chrome()
driver.get(r'https://weibo.com/')
time.sleep(10)
driver.find_element_by_xpath(r'/html/body/div[1]/div[1]/div/div[1]/div/div/div[3]/div[2]/ul/li[3]/a').click()
# driver.find_element_by_xpath(r'//a[@class="S_txt1" and @node-type="loginBtn"]').click()
time.sleep(5)
driver.find_element_by_xpath(r'/html/body/div[4]/div[2]/div[3]/div[1]/a[2]').click()
time.sleep(3)

In [3]:
driver.find_element_by_xpath(r'/html/body/div[1]/div/div[2]/div[3]/div[1]/div/div[2]/div[1]/div[5]/div[1]/a').click()
# driver.find_element_by_xpath(r'//a[@page_id="102803_ctg1_1760_-_ctg1_1760"]').click()
time.sleep(3)
driver.find_element_by_xpath(r'/html/body/div[1]/div/div/div[2]/div[2]/div[1]/div/div/div/div/div[13]/a').click()
# driver.find_element_by_xpath(r'//a[@href="/102803"]').click()
time.sleep(3)

In [4]:
print(driver.page_source[:1000]) #输出获取的link

<html><head>
<meta charset="utf-8">
<meta name="viewport" content="initial-scale=1,minimum-scale=1">
<link rel="dns-prefetch" href="//img.t.sinajs.cn/">
<link rel="dns-prefetch" href="//img1.t.sinajs.cn/">
<link rel="dns-prefetch" href="//js.t.sinajs.cn/">
<link rel="dns-prefetch" href="//js1.t.sinajs.cn/">
<link rel="dns-prefetch" href="//js2.t.sinajs.cn/">
<link rel="dns-prefetch" href="//biz.weibo.com/">
<link rel="dns-prefetch" href="//beacon.sina.com.cn/">
<link rel="dns-prefetch" href="//rs.sinajs.cn/">
<link rel="dns-prefetch" href="//tp1.sinaimg.cn/">
<link rel="dns-prefetch" href="//tp2.sinaimg.cn/">
<link rel="dns-prefetch" href="//tp3.sinaimg.cn/">
<link rel="dns-prefetch" href="//tp4.sinaimg.cn/">
<link rel="dns-prefetch" href="//ww1.sinaimg.cn/">
<link rel="dns-prefetch" href="//ww2.sinaimg.cn/">
<link rel="dns-prefetch" href="//ww3.sinaimg.cn/">
<link rel="dns-prefetch" href="//ww4.sinaimg.cn/">

<link rel="mask-icon" sizes="any" href="//img.t.sinajs.cn/t6/style/images/ap

In [6]:
#执行js 获取窗口滚动条的长度 在滚动到最底端
def scroll2bottom(driver):
    driver.execute_script(r""" 
        (function () { 
            var y = document.body.scrollHeight;
            window.scroll(0, y);
        })(); 
        """)

In [7]:
#一共滚动5次
for i in range(5):
    scroll2bottom(driver)
    time.sleep(3)

In [8]:
driver.find_element_by_link_text('查看更多c').click() #模拟点击查看更多

In [16]:
# 刷新页面 继续滚动 找到所有查看更多
driver.refresh()
time.sleep(3)
for i in range(5):
    for j in range(5):
        scroll2bottom(driver)
        time.sleep(3)
    driver.find_element_by_link_text('查看更多c').click()
    time.sleep(3)

In [12]:
#将滚动条拖到最上
def scroll2top(driver):
    driver.execute_script(r""" 
        (function () {
            window.scrollTo(0, 0)
        })(); 
        """)

In [13]:
#点击所有查看更多
scroll2top(driver)
time.sleep(3)
tags = driver.find_elements_by_link_text('展开全文c')
for tag in tags:
    tag.click()
    time.sleep(1)

In [101]:
contents = driver.find_elements_by_xpath(r'//div[@class="WB_text W_f14"]') #使用xpath找到所要的标签
contents[0].get_attribute('innerHTML') #找到innerHTML属性

'                                                                    大家好，2020真的是讓大家很辛苦的一年，所以一直在找尋我覺得適合的時機告訴大家。今天，我想與大家分享一件好事。<br><br>是的，我现在很幸福，已經有了和我一起共度下半生的另一半，並正式展開我們的家庭生活啦！<br><br>希望大家能給予我們祝福，<br>我們在這裡謝謝你們，<br>也希望你們也能找到屬於自己的幸福 。&nbsp;&nbsp;\u200b\u200b\u200b\u200b...<a target="_blank" href="//weibo.com/1752502537/Jd9nsrL8x" class="WB_text_opt" suda-uatrack="key=original_blog_unfold&amp;value=click_unfold:4531343146616053:1752502537" action-type="fl_unfold" action-data="mid=4531343146616053&amp;is_settop&amp;is_sethot&amp;is_setfanstop&amp;is_setyoudao">展开全文<i class="W_ficon ficon_arrow_down">c</i></a>                                    '

In [14]:
#查看整个页面有多少条微博
contents_full = driver.find_elements_by_xpath(r'//div[@class="WB_text W_f14" and not(@style="display: none;")]')
print('全部博文数量：', len(contents_full))

全部博文数量： 464


In [16]:
contents_full[0].get_attribute('innerHTML') #获取属性

'                                                                    <a target="_blank" render="ext" suda-uatrack="key=topic_click&amp;value=click_topic" class="a_topic" extra-data="type=topic" href="//s.weibo.com/weibo?q=%23%E7%8E%8B%E4%B8%80%E5%8D%9A%E4%B8%8A%E5%A5%BD%E4%BD%B3%E7%94%B0%E5%9B%AD%E8%96%AF%E7%89%87%23&amp;from=default">#王一博上好佳田园薯片#</a><br>今天起我就是<a target="_blank" render="ext" extra-data="type=atname" href="//weibo.com/n/%E4%B8%8A%E5%A5%BD%E4%BD%B3%E5%AE%98%E6%96%B9%E5%BE%AE%E5%8D%9A?from=feed&amp;loc=at" usercard="name=上好佳官方微博">@上好佳官方微博</a> 田园薯片代言人了。薯片吃着，味道品着，不可描述的好吃！对了，去oishi上好佳旗舰店，还有惊喜。<a target="_blank" render="ext" suda-uatrack="key=topic_click&amp;value=click_topic" class="a_topic" extra-data="type=topic" href="//s.weibo.com/weibo?q=%23%E4%B8%8A%E5%A5%BD%E4%BD%B3%E8%96%AF%E6%88%91%E6%9C%89%E8%BF%99%E5%91%B3%23&amp;from=default">#上好佳薯我有这味#</a>！<a suda-uatrack="key=tblog_card&amp;value=click_title:4541069515305099:1034-video:1034%3A4540412082192404:page_102803_home:5

In [20]:
#创建csv文件保存抓取的微博
if not os.path.exists('data/result_Weibo_hot_24h.csv'):
    headers = ['content']
    with open('data/result_Weibo_hot_24h.csv', 'w', newline = '', encoding = 'utf8') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
with open('data/result_Weibo_hot_24h.csv', 'a', newline = '', encoding = 'utf8') as f:
    f_csv = csv.writer(f)
    for content in contents_full:
        f_csv.writerow([content.get_attribute('innerHTML').strip()])

In [2]:
#用pandas包进行去重
pd.read_csv('data/result_Weibo_hot_24h.csv', encoding = 'utf8').drop_duplicates(keep = 'first')

,content
0,我 到底是什么？<br> <br> <br>自创造101以来，网络上关于我的词条总少不了「丑...
1,就这样背后骂的还被录音的人，可见在公司说话到底是有多么的让人恶心？开公司？ 当老板？赚钱？你...
2,"<a target=\""_blank\"" render=\""ext\"" suda-uatra..."
3,"【<a target=\""_blank\"" render=\""ext\"" suda-uatr..."
4,"<a target=\""_blank\"" render=\""ext\"" suda-uatra..."
...,...
3237,"就是死了啊！ <a suda-uatrack=""key=tblog_card&amp;val..."
3238,【关于“真”】<br>真，包括真实、真诚；“真诚”是我们的心、精神、情感，是每个人最向往的一...
3239,姿态：你们知道我最近为什么...很经常开播比较晚吗？给你们小小透露一下…<br>(点开两个文...
3240,"第三届 <a target=""_blank"" render=""ext"" suda-uatra..."


In [115]:
#设置定时 到时间停止抓取
def compare_time(gtime):
    d1 = time.time()
    d2 = time.mktime(time.strptime(gtime, "%Y-%m-%d %H:%M:%S"))
    delta = d1 - d2
    if delta <= 0:
        print('开始爬取，距离爬取结束还有{}分钟。'.format(-delta / 60))
        return True
    else:
        print('已到达指定爬取结束时间，爬取结束。')
        return False
if not os.path.exists('data/result_Weibo_hot_1h.csv'):
    headers = ['content']
    with open('data/result_Weibo_hot_1h.csv', 'w', newline = '', encoding = 'utf8') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
#开始抓取
driver = webdriver.Chrome()
driver.get(r'https://weibo.com/')
time.sleep(15)
driver.find_element_by_xpath(r'/html/body/div[1]/div[1]/div/div[1]/div/div/div[3]/div[2]/ul/li[3]/a').click()
# driver.find_element_by_xpath(r'//a[@class="S_txt1" and @node-type="loginBtn"]').click()
time.sleep(5)


driver.find_element_by_xpath(r'/html/body/div[4]/div[2]/div[3]/div[3]/div[1]/input').send_keys("myusername")
# driver.find_element_by_xpath(r'//div[@class="item username input_wrap"]/input').send_keys("myusername")
time.sleep(3)
driver.find_element_by_xpath(r'/html/body/div[4]/div[2]/div[3]/div[3]/div[2]/input').send_keys("mypassword")
# driver.find_element_by_xpath(r'//div[@class="item password input_wrap"]/input').send_keys("mypassword")
time.sleep(3)
driver.find_element_by_xpath(r'/html/body/div[4]/div[2]/div[3]/div[3]/div[6]/a').click()
# driver.find_element_by_xpath(r'//a[@class="W_btn_a btn_34px"]').click()
time.sleep(120)


# driver.find_element_by_xpath(r'/html/body/div[4]/div[2]/div[3]/div[1]/a[2]').click()
# time.sleep(120)


driver.find_element_by_xpath(r'/html/body/div[1]/div/div[2]/div[3]/div[1]/div/div[2]/div[1]/div[5]/div[1]/a').click()
# driver.find_element_by_xpath(r'//a[@page_id="102803_ctg1_1760_-_ctg1_1760"]').click()
time.sleep(3)
driver.find_element_by_xpath(r'/html/body/div[1]/div/div/div[2]/div[2]/div[1]/div/div/div/div/div[13]/a').click()
# driver.find_element_by_xpath(r'//a[@href="/102803"]').click()
time.sleep(3)
driver.find_element_by_xpath(r'/html/body/div[1]/div/div/div[2]/div[2]/div[2]/div[1]/div/div/div[2]/div/ul/li[3]/a').click()
# driver.find_element_by_xpath(r'//a[@href="/102803?pids=Pl_Core_NewMixFeed__3&feed_sort=102803_ctg1_9999_-_ctg1_9999&feed_filter=102803_ctg1_9999_-_ctg1_9999#Pl_Core_NewMixFeed__3"]').click()
time.sleep(3)

#抓取逻辑
gtime = '2020-08-25 8:40:0'
while compare_time(gtime):
    driver.refresh()
    time.sleep(5)
    try:
        for i in range(5):
            for j in range(5):
                scroll2bottom(driver)
                time.sleep(3)
            driver.find_element_by_link_text('查看更多c').click()
            time.sleep(3)
        scroll2top(driver)
        time.sleep(3)
        tags = driver.find_elements_by_link_text('展开全文c')
        for tag in tags:
            tag.click()
            time.sleep(1)
    except:
        print('error, refresh')
        continue
    else:
        contents_full = driver.find_elements_by_xpath(r'//div[@class="WB_text W_f14" and not(@style="display: none;")]')
        with open('data/result_Weibo_hot_1h.csv', 'a', newline = '', encoding = 'utf8') as f:
            f_csv = csv.writer(f)
            for content in contents_full:
                f_csv.writerow([content.get_attribute('innerHTML').strip()])
        time.sleep(3600)
driver.quit()

已到达指定爬取结束时间，爬取结束。


In [3]:
pd.read_csv('data/result_Weibo_hot_1h.csv', encoding = 'utf8').drop_duplicates(keep = 'first')

,content
0,"【<a target=\""_blank\"" render=\""ext\"" suda-uatr..."
1,昨天的几乎所有舆论钧正平，最高检，半月谈、人民网、人民日报、光明日报、瞭望、新华网、环球网、...
2,"【<a target=\""_blank\"" render=\""ext\"" suda-uatr..."
3,“买包混入贵妇圈最终逆袭”的可行性版本，我觉得大概应该是这样的：<br><br>顾佳砸锅卖铁...
4,"<a suda-uatrack=\""key=tblog_card&amp;value=cli..."
...,...
18399,盘尾市场动静不大了，3184点的大级别低点稳住了，但盘尾反弹力度不够，下面就看周末消息面了，...
18400,被咨询者震惊到了，我必须来做一次小普查。<br>你们和男友、丈夫相处时，会担心因为相处时间久...
18401,"<a suda-uatrack=""key=tblog_card&amp;value=clic..."
18402,【探访杭州失踪女子小区：二度出现网红直播，家属蹲化粪池井盖旁痛哭】7月23日晚，在经过数次辟...


In [18]:
# labels= ['社会', '国际', '科技', '科普', '数码', '财经', '股市', '明星', '综艺', '电视剧', '电影', 
#          '音乐', '汽车', '体育', '运动健身', '健康', '瘦身', '养生', '军事', '历史', '美女模特', '摄影',
#          '情感', '搞笑', '辟谣', '正能量', '政务', '游戏', '旅游', '育儿', '校园', '美食', '房产', '家居', 
#          '星座', '读书', '三农', '设计', '艺术', '时尚', '美妆', '动漫', '宗教', '萌宠', '婚庆', '法律', '舞蹈']
#创建标签列表
labels= ['社会', '国际', '科技',  '财经', '明星', '音乐', '汽车', '体育', '健康', '军事', 
         '历史', '情感', '搞笑',  '游戏', '旅游', '校园', '美食', '艺术', '时尚', '动漫']

In [20]:
#获取得所有查看更多 并模拟点击
scroll2top(driver)
time.sleep(3)
driver.find_element_by_link_text(labels[0]).click()
time.sleep(3)
for i in range(5):
    for j in range(5):
        scroll2bottom(driver)
        time.sleep(3)
    driver.find_element_by_link_text('查看更多c').click()
    time.sleep(3)
scroll2top(driver)
time.sleep(3)
tags = driver.find_elements_by_link_text('展开全文c')
for tag in tags:
    tag.click()
    time.sleep(1)

In [26]:
#输出各类标签微博的个数
contents_full = driver.find_elements_by_xpath(r'//div[@class="WB_text W_f14" and not(@style="display: none;")]')
print('{}类博文数量：'.format(labels[0]), len(contents_full))

社会类博文数量： 460


In [6]:
#同上
def scroll(driver):
    driver.execute_script(r""" 
        (function () { 
            window.scrollTo(0, 200) //使窗口定位至(0, 200)
        })(); 
        """)
if not os.path.exists('data/result_Weibo_with_label.csv'):
    headers = ['content', 'label']
    with open('data/result_Weibo_with_label.csv', 'w', newline = '', encoding = 'utf8') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
i = 0
while i < len(labels):
    try:
        time.sleep(5)
        if i < 17:
            scroll2top(driver)
        else:
            scroll(driver)
        time.sleep(2)
        driver.find_element_by_link_text(labels[i]).click()
        time.sleep(5)
        try:
            for j in range(5):
                for k in range(5):
                    scroll2bottom(driver)
                    time.sleep(3)
                driver.find_element_by_link_text('查看更多c').click()
                time.sleep(3)
        except:
            pass
        scroll2top(driver)
        time.sleep(2)
        tags = driver.find_elements_by_link_text('展开全文c')
        for tag in tags:
            tag.click()
            time.sleep(1)
        contents_full = driver.find_elements_by_xpath(r'//div[@class="WB_text W_f14" and not(@style="display: none;")]')
#         print()
        print('{0}类博文数量：{1}'.format(labels[i], len(contents_full)), end = '')
        with open('data/result_Weibo_with_label.csv', 'a', newline = '', encoding = 'utf8') as f:
            f_csv = csv.writer(f)
            for content in contents_full:
                f_csv.writerow([content.get_attribute('innerHTML').strip(), labels[i]])
    except:
#         print('error, refresh')
        print('*', end = '')
        driver.refresh()
        continue
    else:
        i += 1
        print()


社会类博文数量：462*
国际类博文数量：133
科技类博文数量：458
财经类博文数量：457
明星类博文数量：464
音乐类博文数量：360
汽车类博文数量：356
体育类博文数量：358
健康类博文数量：269
军事类博文数量：359
历史类博文数量：292
情感类博文数量：337
搞笑类博文数量：320
游戏类博文数量：247
旅游类博文数量：224*
校园类博文数量：462
美食类博文数量：461
艺术类博文数量：465
时尚类博文数量：339
动漫类博文数量：433

In [21]:
driver.quit()

In [4]:
#保存去重
Weibo_with_label = pd.read_csv('data/result_Weibo_with_label.csv', encoding = 'utf8').drop_duplicates(['content'], keep = 'first').reset_index(drop = True)
Weibo_with_label

,content,label
0,全球每年生产近1000亿件衣服，但最终85%的纺织品都要被丢弃和填埋，可持续的时尚行业将为地...,社会
1,"<a target=""_blank"" render=""ext"" suda-uatrack=""...",社会
2,"<a target=""_blank"" render=""ext"" suda-uatrack=""...",社会
3,"【<a target=""_blank"" render=""ext"" suda-uatrack=...",社会
4,"【<a target=""_blank"" render=""ext"" suda-uatrack=...",社会
...,...,...
16194,"<a target=""_blank"" render=""ext"" suda-uatrack=""...",动漫
16195,"<a target=""_blank"" render=""ext"" suda-uatrack=""...",动漫
16196,"<a target=""_blank"" render=""ext"" suda-uatrack=""...",动漫
16197,"<a target=""_blank"" render=""ext"" suda-uatrack=""...",动漫


In [6]:
Weibo_with_label.to_csv('data/result_Weibo_with_label.csv', index = 0)

In [1]:
#使用requests库调用微博接口获取数据 提取json中所要的数据 保存到csv文件中
import requests
import json
import os
import csv
page = 1
contents = []
while page <= 13000:
    try:
        t = requests.get(r'https://m.weibo.cn/api/container/getIndex?type=uid&value=2656274875&containerid=1076032656274875&page={}'.format(page), 
                        {
                         'user-agent' : r'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Mobile Safari/537.36'
                        })
        if page == 1:
            for i in range(1, len(t.json().get('data').get('cards'))):
                contents.append(t.json().get('data').get('cards')[i].get('mblog').get('raw_text'))
        else:
            for card in t.json().get('data').get('cards'):
                contents.append(card.get('mblog').get('raw_text'))
    except:
        print('error at page {} {}'.format(page, str(t)))
        continue
    else:
        if page % 1000 == 0:
            print('page {} done'.format(page))
        page += 1
if not os.path.exists('data/result_Weibo_CCTVNEWS.csv'):
    headers = ['content']
    with open('data/result_Weibo_CCTVNEWS.csv', 'w', newline = '', encoding = 'utf8') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
with open('data/result_Weibo_CCTVNEWS.csv', 'a', newline = '', encoding = 'utf8') as f:
    f_csv = csv.writer(f)
    for row in contents:
        f_csv.writerow([row])

page 1000 done
page 2000 done
page 3000 done
page 4000 done
page 5000 done
page 6000 done
page 7000 done
page 8000 done
page 9000 done
page 10000 done
page 11000 done
page 12000 done
page 13000 done


In [3]:
pd.read_csv('data/result_Weibo_CCTVNEWS.csv', encoding = 'utf8').drop_duplicates()

,content
0,【24小时不间断！#央视新闻正直播#】今天，@央视新闻 #买遍中国#带货直播打卡贵州，晚20...
1,#买遍中国#【今晚20点！有场很下饭的直播等你来看[馋嘴]】今晚20点，#买遍中国#贵州站！...
2,#云游中国#【走，带你游多彩贵州！[心]】逛一逛飞角重檐相间的青岩古镇，再去猴耳天坑云端漫步...
3,【自损一千伤敌八百 #美国在焦虑啥#[思考]】目前，包括华为在内共有150多家关联公司被美国...
4,【#浙江两单位因懒政怠政被颁蜗牛奖#】近日，浙江缙云县召开了一场没有掌声的“颁奖礼”。县水利...
...,...
112335,【视频：香港南丫岛撞船事故最新情况】最新情况：死亡人数已上升至36人，事故仍有失踪者，已派出...
112336,#指尖上的高速# 北京京藏高速出行方向部分路段车流量大，大广高速密云县出京方向北庄段、六环路...
112337,【日本本州东海岸发生6.2级地震】根据美国地质勘探局地震信息网的消息，当地时间2号早上7点2...
112338,#最in信息#【视频：北京故宫迎来旅游高峰】据记者报道，1日客流比去年同期增长3万多人次，至...
